In [3]:
from omegaconf import OmegaConf
from databases import load_data
import numpy as np

import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import pickle
import gzip
from pathlib import Path

config = OmegaConf.load("./configs/default_config_linearprobe50.yaml")
config.datasets_path = './data' # where you store datasets locally 

for dataset in [ 
                'uc-merced-land-use-dataset',   
                 'flowers-102', 
                 'caltech101',
                 'stanford_cars', 
                 'fgvc-aircraft-2013b',
                 'oxford-iiit-pet'
                   ]:
                #]: #'dtd', 'eurosat' ,

    print(dataset)
    
    config.dataset = dataset
    train_dataset, val_dataset, test_dataset, N = load_data(config, common_corruption=False)
    _, _, common_test_dataset, N = load_data(config, common_corruption=True)

    output_path = "/home/mheuillet/Desktop/processed_dataset/{}_processed.pkl.gz".format(config.dataset)
    with gzip.open(output_path, 'wb') as f:
        pickle.dump({
            "train": train_dataset,
            "val": val_dataset,
            "test": test_dataset,
            "test_common":common_test_dataset,
            "N": N
        }, f)

    print(f"✅ Saved to: {output_path}")

uc-merced-land-use-dataset
✅ Saved to: /home/mheuillet/Desktop/processed_dataset/uc-merced-land-use-dataset_processed.pkl.gz
flowers-102
✅ Saved to: /home/mheuillet/Desktop/processed_dataset/flowers-102_processed.pkl.gz
caltech101
✅ Saved to: /home/mheuillet/Desktop/processed_dataset/caltech101_processed.pkl.gz
stanford_cars
✅ Saved to: /home/mheuillet/Desktop/processed_dataset/stanford_cars_processed.pkl.gz
fgvc-aircraft-2013b
✅ Saved to: /home/mheuillet/Desktop/processed_dataset/fgvc-aircraft-2013b_processed.pkl.gz
oxford-iiit-pet
✅ Saved to: /home/mheuillet/Desktop/processed_dataset/oxford-iiit-pet_processed.pkl.gz


In [23]:
from omegaconf import OmegaConf
from databases import load_data
import numpy as np

import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import pickle
import gzip
from pathlib import Path

config = OmegaConf.load("./configs/default_config_linearprobe50.yaml")
config.datasets_path = './data' # where you store datasets locally 

for dataset in [ 
                'uc-merced-land-use-dataset',   
                #  'flowers-102', 
                #  'caltech101',
                #  'stanford_cars', 
                #  'fgvc-aircraft-2013b',
                #  'oxford-iiit-pet'
                   ]:
                #]: #'dtd', 'eurosat' ,

    print(dataset)
    
    config.dataset = dataset
    
    path = "/home/mheuillet/Desktop/processed_dataset/{}_processed.pkl.gz".format(config.dataset)
    with gzip.open(path, 'rb') as f:
        data = pickle.load(f)

    print(f"✅ Saved to: {output_path}")

uc-merced-land-use-dataset
✅ Saved to: /home/mheuillet/Desktop/processed_dataset/oxford-iiit-pet_processed.pkl.gz


In [29]:
data['val'].subset

In [ ]:
import pickle
import gzip
import tarfile
import os
import io
import json
import csv
from pathlib import Path
from PIL import Image
import torch
import zstandard as zstd

for dataset in [ 
                # 'uc-merced-land-use-dataset',   
                 'flowers-102', 
                 'caltech101',
                 'stanford_cars', 
                 'fgvc-aircraft-2013b',
                 'oxford-iiit-pet'
                   ]:

    # === CONFIG ===
    pkl_path = "/home/mheuillet/Desktop/processed_dataset/{}_processed.pkl.gz".format(dataset)
    output_tar_path = "/home/mheuillet/Desktop/processed_dataset/{}_processed.tar.zst".format(dataset)
    tmp_dir = Path("/home/mheuillet/Desktop/processed_dataset/tmp_extracted")  # temp folder before archiving

    # === STEP 1: Load pkl.gz ===
    with gzip.open(pkl_path, "rb") as f:
        data = pickle.load(f)

    splits = ["train", "val", "test", "test_common"]
    N = data.get("N", None)

    # === STEP 2: Write files to temporary directory ===
    if tmp_dir.exists():
        import shutil
        shutil.rmtree(tmp_dir)
    tmp_dir.mkdir(parents=True)

    metadata = {"N": N, "splits": {}}

    for split in splits:
        dataset = data[split]
        split_dir = tmp_dir / split
        split_dir.mkdir()

        labels_path = split_dir / "labels.csv"
        with open(labels_path, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["filename", "label"])

            for i, (img, label) in enumerate(dataset):
                filename = f"{i:05d}.png"
                img_path = split_dir / filename

                # Save image
                if isinstance(img, torch.Tensor):
                    img = transforms.ToPILImage()(img)
                img.save(img_path)

                # Save label
                writer.writerow([filename, label])

            metadata["splits"][split] = {"count": i + 1}

    # === STEP 3: Create tar.zst archive ===
    cctx = zstd.ZstdCompressor(level=3)
    with open(output_tar_path, "wb") as f_out:
        with cctx.stream_writer(f_out) as zst_stream:
            with tarfile.open(fileobj=zst_stream, mode="w|") as tar:
                for root, dirs, files in os.walk(tmp_dir):
                    for file in files:
                        fullpath = os.path.join(root, file)
                        arcname = os.path.relpath(fullpath, start=tmp_dir)
                        tar.add(fullpath, arcname=arcname)

                # Include metadata.json
                meta_bytes = json.dumps(metadata, indent=2).encode("utf-8")
                tarinfo = tarfile.TarInfo(name="metadata.json")
                tarinfo.size = len(meta_bytes)
                tar.addfile(tarinfo, io.BytesIO(meta_bytes))

    print(f"✅ Saved: {output_tar_path}")


In [ ]:
### don't pay attention to this cell

# EuroSAT: seulement 8 runs on complete et tous les autres OOM
# Caltech: 26 runs ok, 
# Stanford Cars: 34 ok
# Aircraft: 26 ok
# DTD : 26 ok
# Oxford pet: 26 ok
# Flowers: 26 ok

stanford_cars = 6922 + 1222 + 8041 
caltech101 = 5899 + 1042 + 1736 
dtd = 1880 + 1880 + 1880 
eurosat = 18360 + 3240 + 5400 
fgvcaircraft2013b =  3334 + 3333 + 3333 
flowers102 = 1020 + 1020 + 6149 
oxfordiiitpet = 3128 + 552 + 3669 

print(eurosat, caltech101, dtd, fgvcaircraft2013b, flowers102, oxfordiiitpet, stanford_cars)

from PIL import Image
import torchvision.datasets as datasets

# Initialize the Caltech101 dataset without transformations
caltech101_sample = datasets.Caltech101(root='./data', download=False)
print(len(caltech101_sample))
for i in range( len(caltech101_sample) ) :
    image, label = caltech101_sample[i]

    # Check the image mode
    # print(f"Image Mode: {image.mode}")  # Expected: 'RGB'
    if image.mode != 'RGB':
        print('issue')